Сейчас я хочу оба вида признаков (из КаунтВекторайзера и те, что я сама выделила) засунуть в классификатор. Я надеюсь, что совместный результат будет лучше.

In [115]:
import re
import nltk
import string
import itertools
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize

from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from gensim.models.keyedvectors import FastTextKeyedVectors
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, homogeneity_score, completeness_score, v_measure_score


In [116]:
df_features = pd.read_csv('gk4z_features.csv')
df_features.tail()

,id,text,likes,label,len_char,punct,punct_count,punct%,tokens,tokens_count,tokens%,smiles,smiles_count,smiles%,numbers_count,numbers%,latin_count,latin%
56195,37,ходить кругами сабатон что за мем,23,post,33,NaN,0,0.0,"['ходить', 'кругами', 'сабатон', 'что', 'за', ...",6,0.181818,NaN,0,0.0,0,0.0,0,0.0
56196,33,отель арарат снять номер,63,post,24,NaN,0,0.0,"['отель', 'арарат', 'снять', 'номер']",4,0.166667,NaN,0,0.0,0,0.0,0,0.0
56197,31,рэй джаст арена охранники,57,post,25,NaN,0,0.0,"['рэй', 'джаст', 'арена', 'охранники']",4,0.160000,NaN,0,0.0,0,0.0,0,0.0
56198,20,жирный из аскинг александрия куда делся,90,post,39,NaN,0,0.0,"['жирный', 'из', 'аскинг', 'александрия', 'куд...",6,0.153846,NaN,0,0.0,0,0.0,0,0.0
56199,1,что такое гиг,79,post,13,NaN,0,0.0,"['что', 'такое', 'гиг']",3,0.230769,NaN,0,0.0,0,0.0,0,0.0


In [104]:
df_features.text.fillna('', inplace=True) # заменяем нан на пустую строку

##### Сначала делаю признаки CountVectorizer+TruncatedSVD.

TruncatedSVD я пользуюсь, чтобы вектор признаков не был гигантским.


In [8]:
vectorizer = CountVectorizer(tokenizer=wordpunct_tokenize)
bow = vectorizer.fit_transform(df_features.text)

In [70]:
svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
svd.fit(bow)
bow_svd = svd.transform(bow)
# я пробовала с разным количеством n_components, оставила с 500, так лучший результат получается.

In [24]:
type(bow_svd)

numpy.ndarray

,id,text,likes,label,len_char,punct,punct_count,punct%,tokens,tokens_count,tokens%,smiles,smiles_count,smiles%,numbers_count,numbers%,latin_count,latin%,vec
0,76000,#конкурс@the4gkz \r\r\nДля вашего удобства соб...,17,post,503,"#@.:•|-.—:./:./•|-.—/:./:://./|://./•|.—:./:.,!",47,0.093439,"['#', 'конкурс', '@', 'the4gkz', 'Для', 'вашег...",81,0.161034,NaN,0,0.000000,23,0.045726,116,0.230616,"[7.8431077444484, 11.778166764394879, 1.034311..."
1,76914,Уже сегодня ночью (по МСК) начнётся один из са...,34,post,669,"()—.—()!..,!(),—(,).:://..//",28,0.041854,"['Уже', 'сегодня', 'ночью', '(', 'по', 'МСК', ...",111,0.165919,NaN,0,0.000000,0,0.000000,49,0.073244,"[6.499004644916794, 3.0822661997487635, 0.7564..."
2,76917,"Ого, Нина Кравиц в лайн-апе😲",0,comment,28,",-",2,0.071429,"['Ого', ',', 'Нина', 'Кравиц', 'в', 'лайн-апе😲']",6,0.214286,😲,1,0.035714,0,0.000000,0,0.000000,"[1.1437397018609134, -0.2865937475635932, 0.07..."
3,76920,"[id17823823|Анастасия], с новым крутым шоу от ...",0,comment_to_comment,56,"[|],",4,0.071429,"['[', 'id17823823|Анастасия', ']', ',', 'с', '...",11,0.196429,NaN,0,0.000000,8,0.142857,2,0.035714,"[0.21056163877449977, 0.31816780662500105, 1.5..."
4,76922,Скину расписание на сегодня сюда. Время москов...,0,comment,687,..:......&....:..........:......&.....,38,0.055313,"['Скину', 'расписание', 'на', 'сегодня', 'сюда...",112,0.163028,NaN,0,0.000000,127,0.184862,332,0.483261,"[15.280310689921265, 24.73979390550555, -8.681..."
5,76933,"[id18399319|Anastasia], неправда, в Москве вре...",0,comment_to_comment,85,"[|],,,",6,0.070588,"['[', 'id18399319|Anastasia', ']', ',', 'непра...",17,0.200000,NaN,0,0.000000,8,0.094118,11,0.129412,"[2.0202150115404476, -0.8198600461492136, 1.71..."
6,76929,"Трансляция, конечно, моё почтение. Классно выг...",1,comment,66,",,..",4,0.060606,"['Трансляция', ',', 'конечно', ',', 'моё', 'по...",12,0.181818,NaN,0,0.000000,0,0.000000,0,0.000000,"[2.3453349784004978, 0.41370213776248715, -0.6..."
7,76930,"Моё увожение Блэкпинк, которые даже с живыми и...",1,comment,83,",.",2,0.024096,"['Моё', 'увожение', 'Блэкпинк', ',', 'которые'...",14,0.168675,🤘,1,0.012048,0,0.000000,0,0.000000,"[1.3026134559347782, 0.18898780739635715, -0.3..."
8,76934,"Часть пропустила, конец сета у SG Lewis был кл...",0,comment,307,",.-,,.-,-",9,0.029316,"['Часть', 'пропустила', ',', 'конец', 'сета', ...",56,0.182410,🙀🔥,2,0.006515,4,0.013029,40,0.130293,"[5.535550244671003, 0.030437951163541095, -0.4..."
9,76935,"сейчас посмотрела лайв Rufus Du Sol, очень бод...",0,comment,134,",.,",3,0.022388,"['сейчас', 'посмотрела', 'лайв', 'Rufus', 'Du'...",21,0.156716,NaN,0,0.000000,0,0.000000,10,0.074627,"[1.9784572778914467, -0.3105016366509472, -0.3..."


##### Теперь делаю аррей с оставшимися признаками

In [22]:
feature_names = ['likes', 'len_char', 'punct_count',
       'punct%', 'tokens_count', 'tokens%', 'smiles_count',
       'smiles%', 'numbers_count', 'numbers%', 'latin_count', 'latin%']

In [23]:
a = df_features.filter(items = feature_names)
a = a.values

In [28]:
a.shape

(56200, 12)

In [ ]:
bow_svd

##### Конкатенирую признаки

In [71]:
array_to_use = np.concatenate((bow_svd, a), axis=1)

In [72]:
array_to_use.shape

(56200, 512)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(array_to_use, df_features['label'])

In [50]:
# 500
lr = LogisticRegression()
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

                    precision    recall  f1-score   support

           comment       0.89      0.97      0.93      9449
comment_to_comment       0.50      0.19      0.27      1148
              post       0.98      0.96      0.97      3453

       avg / total       0.88      0.90      0.89     14050



Общий результат лучше, но комменты второго уровня не особо хорошо определились.

Для сравнения: то же самое, но только на признаках countVectorizer+trancatedSVD, без тех признаков, что я сама выделяла:
общий результат похуже, но комменты к комментам кое-как определились:

In [32]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow, df_features['label'])

In [33]:
lr = LogisticRegression()
clf = lr.fit(X_train_bow, y_train_bow)
print(classification_report(y_test_bow, clf.predict(X_test_bow)))

                    precision    recall  f1-score   support

           comment       0.87      0.94      0.90      9485
comment_to_comment       0.66      0.41      0.51      1172
              post       0.89      0.80      0.84      3393

       avg / total       0.85      0.86      0.85     14050



Попробую стандартизировать получившийся совместный аррей:

In [73]:
standardized_array_to_use = scale(array_to_use)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(standardized_array_to_use, df_features['label'])

In [74]:
#500
lr = LogisticRegression()
clf = lr.fit(X_train_s, y_train_s)
print(classification_report(y_test_s, clf.predict(X_test_s)))

                    precision    recall  f1-score   support

           comment       0.90      0.97      0.93      9411
comment_to_comment       0.53      0.20      0.29      1141
              post       0.97      0.96      0.96      3498

       avg / total       0.89      0.90      0.89     14050



Ну как-то не фонтан.Комменты второго уровня продолжают игнориться.